In [1]:
# pip install python-dotenv
# pip install psycopg2-binary
# pip install selenium
# pip install selenium-manager

In [ ]:
import os
import requests
from dotenv import load_dotenv
import psycopg2
import re
import time
from db_connect import db_connect
from model import save_cafe_db,fetch_all_cafe,cafe_region_db
from congestion import compute_congestion_score



#env 파일 환경변수 로드
# load_dotenv() 

# # KaKao API 설정
# KAKAO_API_KEY = os.getenv("KAKAO_REST_API_KEY")
# headers = { "Authorization": f"KakaoAK {KAKAO_API_KEY}" }
# url = "https://dapi.kakao.com/v2/local/search/keyword.json"

In [ ]:
# 지역 별 카페 DB에 저장
cafe_region_db()

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

cur, conn = db_connect()

# 크롤링
def crawl_place_info(place_url):
    
    
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    try:
        driver=webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)
        driver.get(place_url)
        time.sleep(5)
        
        print("브라우저 실행 성공")
        wait = WebDriverWait(driver,12)
    
        blog_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'a.link_blog, a.link_review')))
        text = blog_element.get_attribute("textContent")
        if text:
            print("블로그 텍스트:",text.strip())
            match = re.search(r'(블로그|후기|리뷰)\s*(\d+)',text.strip())
            if match:
                print("크롤링 성공",match.group(2))
                return int(match.group(2))
            else:
                print("리뷰 없음")
                return 0
        else:
            print("정규식 불일치 - 리뷰 없음")
            return 0
    except Exception as e:
        print("크롤링 실패",e)
        return None
    
    finally:
        driver.quit()

cur.execute("SELECT kakao_id, place_url FROM cafe")
rows = cur.fetchall()

for cafe_id, place_url in rows:
    review_count = crawl_place_info(place_url)
    if not review_count:
        reiview_count = 0
    print(f"[{cafe_id}] 블로그 리뷰 수: {review_count}")
    
    # 블로그 리뷰 기준으로 혼잡도 계산
    congestion_score = compute_congestion_score(review_count)
    
    cur.execute("""
        UPDATE cafe
        SET review_count = %s,
            congestion_score = %s
        WHERE kakao_id = %s
        """,(review_count,congestion_score,cafe_id))
    print(f"[{cafe_id}] DB 저장 완료")
    print(f"혼잡도 : {congestion_score}")
    conn.commit()
    
    time.sleep(1)
    
cur.close()
conn.close()

DB connect 성공
브라우저 실행 성공
블로그 텍스트: 후기50개
크롤링 성공 50
[27413263] 블로그 리뷰 수: 50
[27413263] DB 저장 완료
혼잡도 : 4
브라우저 실행 성공
블로그 텍스트: 후기19개
크롤링 성공 19
[1259936409] 블로그 리뷰 수: 19
[1259936409] DB 저장 완료
혼잡도 : 2
브라우저 실행 성공
블로그 텍스트: 후기162개
크롤링 성공 162
[7948829] 블로그 리뷰 수: 162
[7948829] DB 저장 완료
혼잡도 : 5
브라우저 실행 성공
블로그 텍스트: 후기52개
크롤링 성공 52
[26395435] 블로그 리뷰 수: 52
[26395435] DB 저장 완료
혼잡도 : 4
브라우저 실행 성공
블로그 텍스트: 후기22개
크롤링 성공 22
[413137032] 블로그 리뷰 수: 22
[413137032] DB 저장 완료
혼잡도 : 3
브라우저 실행 성공
블로그 텍스트: 후기325개
크롤링 성공 325
[22837049] 블로그 리뷰 수: 325
[22837049] DB 저장 완료
혼잡도 : 7
브라우저 실행 성공
블로그 텍스트: 후기13개
크롤링 성공 13
[27196653] 블로그 리뷰 수: 13
[27196653] DB 저장 완료
혼잡도 : 2
브라우저 실행 성공
블로그 텍스트: 후기201개
크롤링 성공 201
[179244712] 블로그 리뷰 수: 201
[179244712] DB 저장 완료
혼잡도 : 6
브라우저 실행 성공
블로그 텍스트: 후기32개
크롤링 성공 32
[17101526] 블로그 리뷰 수: 32
[17101526] DB 저장 완료
혼잡도 : 3
브라우저 실행 성공
블로그 텍스트: 후기62개
크롤링 성공 62
[20494135] 블로그 리뷰 수: 62
[20494135] DB 저장 완료
혼잡도 : 4
브라우저 실행 성공
블로그 텍스트: 후기183개
크롤링 성공 183
[530432745] 블로그 리뷰 수: 183
[530432745] DB 저장 완료
혼잡도 : 